In [1]:
import requests
import json
import pandas as pd

import os
eia_api_key = os.environ['EIA_API_KEY']

# Fetching Raw Data

In [74]:
## GENERATION

params = {
    'api_key': eia_api_key, 
}
headers = {"X-Params":json.dumps(
    {
        "facets":{"respondent": ["CISO"]},
        'start': '2021-06-01',
        "data": ["value"],
        'end':'2021-06-01T23',
        'frequency':'hourly',
        'sort': [{"column": "period","direction":"asc"}]
})}
response = requests.get(
    "https://api.eia.gov/v2/electricity/rto/fuel-type-data/data/",
    params=params, 
    headers=headers
)
pd.DataFrame(response.json()['response']['data']).to_csv('raw/generation.csv', index=False)

In [75]:
# Capacity
headers = {"X-Params": json.dumps({
    "frequency": "monthly",
    "data": [
        "nameplate-capacity-mw"
    ],
    "facets": {
        "balancing_authority_code": [
            "CISO"
        ]
    },
    "start": "2021-06",
    "end": "2021-06",
    "sort": [
        {
            "column": "period",
            "direction": "desc"
        }
    ],
})}
response = requests.get(
    "https://api.eia.gov/v2/electricity/operating-generator-capacity/data/",
    params=params, 
    headers=headers
)
response.json()
pd.DataFrame(response.json()['response']['data']).to_csv('raw/capacity.csv', index=False)

In [85]:
## Imports/Exports

params = {
    'api_key': eia_api_key, 
}
headers = {"X-Params":json.dumps(
    {
        "facets":{"fromba": ["CISO"]},
        'start': '2021-06-01',
        "data": ["value"],
        'end':'2021-06-01T23',
        'frequency':'hourly',
        'sort': [{"column": "period","direction":"asc"}]
})}
response = requests.get(
    "https://api.eia.gov/v2/electricity/rto/interchange-data/data/",
    params=params, 
    headers=headers
)
pd.DataFrame(response.json()['response']['data']).to_csv('raw/imports_exports.csv', index=False)

In [129]:
## Load Forecast. 

params = {
    'api_key': eia_api_key, 
}
headers = {"X-Params":json.dumps(
    {
        "facets":{"respondent": ["CISO"]},
        'start': '2022-06-01T00',
        'end':'2022-06-01T23',
        "data": ["value"],
        'frequency':'hourly',
        'sort': [{"column": "period","direction":"asc"}]
})}

response = requests.get(
    "https://api.eia.gov/v2/electricity/rto/region-data/data/",
    params=params, 
    headers=headers
)

pd.DataFrame(response.json()['response']['data']).to_csv('raw/load_forecast.csv', index=False)

# Transforming Data

In [4]:
import pandas as pd

GRID_NODE_NAME_TO_ID_MAP = {
    "CISO":"US-WECC-CISO",
    "PACW": "US-WECC-PACW",
    "BANC": "US-WECC-BANC",
    "BPAT": "US-WECC-BPAT",
    "CEN": "MX",
    "LDWP": "US-WECC-LDWP",
    "NEVP": "US-WECC-NEVP",
    "AZPS": "US-WECC-AZPS",
    "WALC": "US-WECC-WALC",
    "TIDC": "US-WECC-TIDC",
    "SRP": "US-WECC-SRP",
    "IID": "US-WECC-IID",
}

In [5]:
# Generation

# Time is in UTC
df = pd.read_csv('raw/generation.csv')
df = pd.pivot_table(
    df,
    values="value",
    index=["respondent", "period"],
    columns="type-name"
).fillna(0).reset_index()

# Rename and reformat to align with ENTSO-E
df['start_datetime'] = pd.to_datetime(df['period']).dt.strftime("%Y-%m-%d %H:%M")
df['end_datetime'] = (pd.to_datetime(df['period'])+pd.Timedelta(hours=1)).dt.strftime("%Y-%m-%d %H:%M")
df.drop(['period'], inplace=True, axis=1)
df = df.rename(columns={
    "respondent":"Grid Node",
    "Coal": "OTHER_COAL",
    "Hydro": "HYDRO",
    "Natural gas": "NATURAL_GAS",
    "Nuclear": "NUCLEAR",
    "Other": "OTHER",
    "Petroleum": "OIL",
    "Solar": "SOLAR",
    "Wind": "WIND"
}
)
df['Grid Node'] = df['Grid Node'].map(GRID_NODE_NAME_TO_ID_MAP)
df['unit'] = 'MWh'
df.to_csv('generation.csv', index=False)

In [9]:
CAPACITY_MAPPING = {
    "Agriculture Byproducts": "BIOMASS",
    "Disillate Fuel Oil": "OTHER_OIL",
    "Electricity used for energy storage": "OTHER",
    "Gaseous Propane": "OTHER_GAS",
    "Geothermal": "GEOTHERMAL",
    "Jet Fuel": "OTHER_OIL",
    "Landfill Gas": "OTHER_GAS",
    "Municipal Solid Waste (All)": "BIOMASS",
    "Natural Gas": "OTHER_GAS",
    "Nuclear": "NUCLEAR",
    "Other": "OTHER",
    "Other Biomass Gases": "BIOMASS",
    "Other Gas": "OTHER_GAS",
    "Petroleum Coke": "OTHER_OIL",
    "Purchased Steam": "OTHER",
    "Refined Coal": "HARD_COAL",
    "Solar": "SOLAR",
    "Waste Heat": "BIOMASS",
    "Water": "HYDRO",
    "Wind": "WIND_ON_SHORE",
    "Wood Waste Solids": "BIOMASS",
}

In [10]:
# Capacity
df = pd.read_csv('raw/capacity.csv')
df = df.groupby(["period","energy-source-desc","balancing_authority_code"]).agg({"nameplate-capacity-mw":"sum"})
df = df.reset_index().rename(columns={
    "nameplate-capacity-mw":"Value",
    "energy-source-desc":"Fuel Type",
    "balancing_authority_code":"Grid Node"
})
df['start_datetime'] = pd.to_datetime(df['period']).dt.strftime("%Y-%m-%d %H:%M")
df['end_datetime'] = (pd.to_datetime(df['period'])+pd.Timedelta(hours=1)).dt.strftime("%Y-%m-%d %H:%M")
df = df.drop(["period"], axis=1)

df['Grid Node'] = df['Grid Node'].map(GRID_NODE_NAME_TO_ID_MAP)
df['Fuel Type'] = df['Fuel Type'].map(CAPACITY_MAPPING)
# Remove fuel types unmatched
df = df[~df['Fuel Type'].isna()]
# Group by grid node, datetime, fuel type
df = df.groupby(["Grid Node", "start_datetime","end_datetime", "Fuel Type"]).agg({"Value":"sum"}).reset_index()
df.loc[:, "unit"] = "MW"
df.to_csv('installed_capacity.csv', index=False)

In [11]:
# Imports/Exports
df = pd.read_csv("raw/imports_exports.csv")
df['start_datetime'] = pd.to_datetime(df['period']).dt.strftime("%Y-%m-%d %H:%M")
df['end_datetime'] = (pd.to_datetime(df['period'])+pd.Timedelta(hours=1)).dt.strftime("%Y-%m-%d %H:%M")
df = df.drop(["period"], axis=1)

df = df.reset_index().rename(columns={"fromba": "Grid Node From", "toba": "Grid Node To", "value": "Value"})
df["Grid Node From"] = df["Grid Node From"].map(GRID_NODE_NAME_TO_ID_MAP)
df["Grid Node To"] = df["Grid Node To"].map(GRID_NODE_NAME_TO_ID_MAP)

final_columns = ["start_datetime", "end_datetime", "Grid Node From", "Grid Node To", "Value"]
df = df[final_columns]
df.loc[:, "unit"] = "MWh"

df.to_csv("imports_exports.csv", index=False)

In [17]:
# Load Forecast
df = pd.read_csv('raw/load_forecast.csv')
df = df[df['type'].isin(['D','DF'])]
df['start_datetime'] = pd.to_datetime(df['period']).dt.strftime("%Y-%m-%d %H:%M")
df['end_datetime'] = (pd.to_datetime(df['period'])+pd.Timedelta(hours=1)).dt.strftime("%Y-%m-%d %H:%M")
df.drop(['period'], inplace=True, axis=1)
df = pd.pivot_table(
    df,
    values="value",
    index=["respondent", "start_datetime","end_datetime"],
    columns="type-name"
).reset_index()
df = df.rename(columns={
    "Day-ahead demand forecast": "Demand Forecast (MW)",
    "Demand": "Actual Total Load (MW)",
    "respondent": "Grid Node",
})
df['Grid Node'] = df['Grid Node'].map(GRID_NODE_NAME_TO_ID_MAP)
df['unit'] = 'MWh'
df = df.rename(columns={'Demand Forecast (MW)':'forecast','Actual Total Load (MW)':'actual'})
df.to_csv('load_forecast.csv',index=False)